<a href="https://colab.research.google.com/github/berry2971/senti-dict-petitions/blob/master/senti_dict_petitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###과제 의의
* '불만'이라는 특정한 감성을 추출해내기란 쉽지 않다. 하지만 국민청원 게시판은 '불만, 원망, 간청'이라는 감성이 전적으로 드러나 있는 게시판이다. 이에 국민청원 게시판의 텍스트 분석을 통해 '불만'의 감성을 가진 단어들을 상당히 추출할 수 있을 것으로 보인다.

###과제 수행 방법
#####스크래핑
* Selenium을 사용하되, 별도의 코드로 실행함. (https://github.com/berry2971/scraper-petitions)

#####분석 대상
* 130641건의 국민청원을 대상으로 함.
* 보건복지: 6317
* 저출산/고령화대책: 960
* 행정: 6673
* 미래: 7589
* 안전/환경: 11460
* 성장동력: 3506
* 기타: 16800
* 일자리: 6884
* 외교/통일/국방: 9385
* 반려동물: 1481
* 교통/건축/국토: 8603
* 경제민주화: 6591
* 농산어촌: 603
* 육아/교육: 6833
*  정치개혁: 19518
* 인권/성평등: 10177
* 문화/예술/체육/언론: 7261

####형태소 분석
* 형태소 분석의 경우, SentencePiece 등 비지도 기반 형태소 분석기를 사용할 수도 있다. 하지만 국민청원의 경우 정부와 국민 사이의 공식적인 소통 창구로서 꽤나 정제된 언어로 작성될 뿐만 아니라, 만약 욕설 등 연구 가치가 있지만 OOV(Out Of Vocabulary)가 발생할 가능성이 높은 어휘이라 할지라도, 대부분 게시판 관리자에 의해 제거된다. 따라서 속도와 성능을 모두 고려하여 Mecab을 사용하여도 충분할 것이라 판단했다.
* Mecab은 일부 형태에 대해 용언과 어미를 붙여서 분석하기 때문에, lovit[1]이 공개한 soylemma 라이브러리를 통해 어간을 분석해낸다. 이는 okt 형태소 분석기의 POS 목록을 따르기 때문에 어미를 관형사형전성어미(ETM), 명사형전성어미(ETN) 등으로 구분할 수 없기에 어미를 누락시킬 수밖에 없다는 문제점을 지닌다. 그러나, 이러한 과정을 거치지 않으면 특정 품사의 출현 빈도를 분석해내는 과정에서, 실제 빈도와 분석 빈도 간의 격차가 커질 것임이 분명하다.

####KNU(케이앤유) 한국어 감성 사전(군산대학교)[2] 사용
* 박상민(군산대학교)·나철원(군산대학교)·최민성(군산대학교)·이다희(군산대학교)· 온병원(군산대학교)(2018), Bi-LSTM 기반의 한국어 감성사전 구축 방안
* KNU 한국어 감성 사전은 형용사뿐만 아니라 부사 등을 포함하고 있어 보다 풍부한 어휘를 지니고 있고, 도메인에서 비교적 자유롭다. 14843개의 어휘로 구성되어 있다.
* 기발표된 KNU 한국어 감성 사전의 부정 극성 단어를 기반으로 하여, 국민청원 게시물에서 Word2Vec을 통해 해당 단어와 유사한 맥락을 지니는 형용사(VA)/부사(MAG)/동사(VV)를 추출해 낸다.
* 일반명사(NNG), 고유명사(NNP)의 경우, 감성이 아니라, 해당 시기의 이슈와 트렌드를 반영하여 분석될 확률이 다분하여 추출 대상에서 제외한다.
* KNU 한국어 감성 사전의 각 단어들에 대해 추출이 진행되기 때문에 중복되는 단어가 발생하는데, 이 경우 가장 높은 코사인 유사도를 단어와 함께 저장한다.

[1] https://github.com/lovit

[2] https://github.com/park1200656/KnuSentiLex

In [66]:
##### Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [0]:
##### Connect with db (raw text)
import sqlite3
con1 = sqlite3.connect('/content/gdrive/My Drive/senti/result_petitions.db')
cur1 = con1.cursor()

In [0]:
##### Connect with db (analyzed text)
con2 = sqlite3.connect('/content/gdrive/My Drive/senti/result_petitions_tag_with_mecab.db')
#con2.execute('CREATE TABLE analysis (TAGGED BLOB, CATEGORY TEXT, AGREES INTEGER)')

In [13]:
##### Install and import pos tagger: mecab
!pip install python-mecab-ko
import mecab
mecab = mecab.MeCab()

### Example
print("Example")
raw = "오늘은 지갑을 집에 두고 왔다."
print(raw)
print(mecab.pos(raw))

Example
오늘은 지갑을 집에 두고 왔다.
[('오늘', 'NNG'), ('은', 'JX'), ('지갑', 'NNG'), ('을', 'JKO'), ('집', 'NNG'), ('에', 'JKB'), ('두', 'VV'), ('고', 'EC'), ('왔', 'VX+EP'), ('다', 'EF'), ('.', 'SF')]


In [12]:
##### Install and import sentence divider: kss(Korean Sentence Splitter by 박상길)
!pip install kss
import kss

### Example
print("Example")
raw = "안녕하세요. 오늘은 날씨가 맑네요! 하늘도 푸르구요! 미세먼지만 없으면ㅠㅠ 좋을텐데ㅠㅠ"
print(raw)
print(kss.split_sentences(raw))

Example
안녕하세요. 오늘은 날씨가 맑네요! 하늘도 푸르구요! 미세먼지만 없으면ㅠㅠ 좋을텐데ㅠㅠ
['안녕하세요.', '오늘은 날씨가 맑네요!', '하늘도 푸르구요!', '미세먼지만 없으면ㅠㅠ 좋을텐데ㅠㅠ']


In [10]:
##### Install and import stem extractor: soylemma(by Lovit)
!pip install soylemma
from soylemma import Lemmatizer
lemmatizer = Lemmatizer()

### Example
print("Example")
print("안녕: {}".format(lemmatizer.lemmatize("안녕")))
print("예쁜: {}".format(lemmatizer.lemmatize("예쁜")))
print("간: {}".format(lemmatizer.lemmatize("간")))

Example
안녕: []
예쁜: [('예쁘다', 'Adjective')]
간: [('가다', 'Verb'), ('갈다', 'Verb')]


In [0]:
########## SPENDS LONG TIME ##########
########## EXECUTE FIRST TIME ONLY ########## (See Line 33)
##### Pos tag and save it into database
import pickle
import re

count = 0

# Pos need to be saved
for row in cur1.execute("SELECT title, content, category, agrees FROM petitions"):
    tagged_sentences = []
    title_and_content = row[0]+"\n"+row[1]
    for sentence in kss.split_sentences(title_and_content):
        temp = []
        tag_res = mecab.pos(sentence)
        for t in tag_res:
            if re.search('\+', t[1]) == None:
                temp.append(t)
            else: # get stem from tagged tokens like ('하신', 'VV+ETM')
                lem = lemmatizer.analyze(t[0])
                if len(lem) != 0:
                    stem = lem[0] # (('예쁘', 'Adjective'), ('ㄴ', 'Eomi'))
                    if stem[0][1] == 'Verb':
                        temp.append((stem[0][0], "VV"))
                    elif stem[0][1] == 'Adjective':
                        temp.append((stem[0][0], "VA"))
        if temp:
            tagged_sentences.append(temp)
    
    # Insert pickled data into database
    data = tagged_sentences
    pdata = pickle.dumps(data)
    # con2.execute('INSERT INTO analysis (TAGGED, CATEGORY, AGREES) VALUES (?, ?, ?)', [pdata, row[2], row[3]])

    # Check progress and Commit database
    count += 1
    if count % 5000 == 0:
        print(count)
        con2.commit()

con2.commit()

In [0]:
def parse_token(token):
    word = re.search("(.*)/(.*)", token).group(1)
    pos = re.search("(.*)/(.*)", token).group(2)
    return (word, pos)

def parse_tagged_word(word):
    lst = word.split(sep = "/")
    return (lst[0], lst[1])

In [16]:
##### Load database
import pickle

con3 = con2
cur3 = con3.cursor()

data_ctg = dict()
categories = ['보건복지', '저출산/고령화대책', '행정', '미래', '안전/환경', '성장동력', '기타', '일자리', '외교/통일/국방', '반려동물', '교통/건축/국토', '경제민주화', '농산어촌', '육아/교육', '정치개혁', '인권/성평등', '문화/예술/체육/언론']
count_doc_categories = []
for i in range(len(categories)):
    count_doc_categories.append(0)
for c in categories:
    data_ctg.update({c:[]})

count = 0
for docs in cur3.execute("SELECT * FROM analysis"):
    # Check progress
    count += 1
    if count % 15000 == 0:
        print(count, end='...')
        print(int(count/130641*100), end='')
        print("%")
    if count == 130641:
        print("130641...100%")

    content = pickle.loads(docs[0])
    category = docs[1]
    agrees = docs[2]

    # Count numbers
    count_doc_categories[categories.index(category)] += 1

    # Convert (안녕, IC) to 안녕/IC
    for sentence in content:
        for i in range(len(sentence)):
            word = sentence[i][0]
            pos = sentence[i][1]
            tagged_word = word+"/"+pos
            sentence[i] = tagged_word
            
    data_ctg[category].append((content, agrees))

print()
for c in categories:
    print("{c}: {num}".format(c=c, num=count_doc_categories[categories.index(c)]))

15000...11%
30000...22%
45000...34%
60000...45%
75000...57%
90000...68%
105000...80%
120000...91%
130641...100%

보건복지: 6317
저출산/고령화대책: 960
행정: 6673
미래: 7589
안전/환경: 11460
성장동력: 3506
기타: 16800
일자리: 6884
외교/통일/국방: 9385
반려동물: 1481
교통/건축/국토: 8603
경제민주화: 6591
농산어촌: 603
육아/교육: 6833
정치개혁: 19518
인권/성평등: 10177
문화/예술/체육/언론: 7261


In [0]:
##### Preprocess data
data_ctg_sentences = dict() # {'보건복지':[['먼저/MAG','가/VV', '아/EF'], ['오늘/NNG', '도/JX']], '일자리':[['성공/NNG', '하/XSV', '았/EP', '나/EF'], ['글쎄/IC']], ...}
categories = ['보건복지', '저출산/고령화대책', '행정', '미래', '안전/환경', '성장동력', '기타', '일자리', '외교/통일/국방', '반려동물', '교통/건축/국토', '경제민주화', '농산어촌', '육아/교육', '정치개혁', '인권/성평등', '문화/예술/체육/언론']
for c in categories:
    data_ctg_sentences.update({c:[]})

for key in data_ctg.keys():
    sentences = []
    for d in data_ctg[key]:
        for sentence in d[0]:
            sentences.append(sentence)
    data_ctg_sentences[key].extend(sentences)

In [0]:
iter = 50
window = 5

In [0]:
##### Model without considering category
whole_sentence = []

for petition in list(data_ctg_sentences.values()): # list(data_ctg_sentences.values())[0][0]: sentence
    for sentence in petition:
        whole_sentence.append(sentence)

from gensim.models import Word2Vec
model_whole = Word2Vec(whole_sentence, size=300, window=window, min_count=10, workers=4, sg=1, iter=iter, sample=0.00001)

In [0]:
##### Save trained model
with open('/content/gdrive/My Drive/senti/models/model_whole.bin', 'wb') as f:
    pickle.dump(model_whole, f)

In [0]:
##### Load trained model
with open('/content/gdrive/My Drive/senti/models/model_whole.bin', 'rb') as f:
    model_whole = pickle.load(f)

In [0]:
##### Model with considering category
from gensim.models import Word2Vec

models = []

count = 0
for v in data_ctg_sentences.values():
    count += 1
    print("{cntt}/17".format(cntt=count))
    models.append(Word2Vec(v, size=300, window=window, min_count=10, workers=4, sg=1, iter=iter, sample=0.00001))

In [0]:
##### Save trained models
for i in range(len(models)):
    ctg_name = categories[i]
    if re.search("/", ctg_name) != None:
        ctg_name = re.sub("/", "+", ctg_name)

    with open('/content/gdrive/My Drive/senti/models/model_{pos}_iter{iter}_window{window}.bin'.format(pos=ctg_name, iter=iter, window=window), 'wb') as f:
        pickle.dump(models[i], f)

In [0]:
##### Load trained models
import re
models = []
for i in range(len(categories)):
    ctg_name = categories[i]
    if re.search("/", ctg_name) != None:
        ctg_name = re.sub("/", "+", ctg_name)
    with open('/content/gdrive/My Drive/senti/models/model_{pos}_iter{iter}_window{window}.bin'.format(pos=ctg_name, iter=iter, window=window), 'rb') as f:
        models.append(pickle.load(f))

Visualize

In [21]:
##### Install Nanum Fonts
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [0]:
obj_model = model_whole # obj_model = models_whole, models[n]
obj_pos = "VV" # obj_pos = "VV", "VA", "MAG"

##### t-SNE: t-Stochastic Neighbor Embedding
import re
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

matplotlib.rcParams["axes.unicode_minus"] = False

vocab = [v for v in list(obj_model.wv.vocab) if parse_tagged_word(v)[1] == obj_pos]
#vocab = obj_model.wv.vocab
X = obj_model[vocab]

tsne = TSNE(n_components = 2)
X_tsne = tsne.fit_transform(X)

##### Show data frame
import pandas as pd
df = pd.DataFrame(X_tsne, index = vocab, columns = ["x", "y"])
df.head()

In [0]:
##### Draw figure
%matplotlib inline

fig = plt.figure()
fig.set_size_inches(30,20)

ax = fig.add_subplot(1, 1, 1)
ax.scatter(df["x"], df["y"])

prop = fm.FontProperties(fname = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf")
for word, pos in list(df.iterrows()):
  ax.annotate(word, pos, fontsize = 12, fontproperties = prop)

plt.show()

In [22]:
##### Read KNU neg_pol_word
knu_negv_words = set()
with open('/content/gdrive/My Drive/senti/neg_pol_word.txt', 'r', encoding = 'utf8') as f:
    knu_negv_file = f.readlines()

one_ej = 0
for i in range(len(knu_negv_file)):
    knu_negv_file[i] = knu_negv_file[i].strip()
    item = knu_negv_file[i]
    if (len(item.split(sep=' ')) == 1): # apply for single ej item only
        one_ej += 1
        pos_res = mecab.pos(item)
        tagged_item = pos_res[0][0]+"/"+pos_res[0][1]
        knu_negv_words.add(tagged_item)
knu_negv_words = list(knu_negv_words) # ['방탕/NNG', '^-^;;/SY', '결핍/NNG', '짖궂/VA', '맛없/VA']
print(one_ej)

4187


In [39]:
##### Get word set
final_word_dict = dict()
final_word_dict_keyset = set()

min_cos_sim = 0.5

for i in range(len(models)):
    print("model_{name}".format(name=categories[i]))
    cur_model = models[i]
    cur_negv_words = []
    for token in knu_negv_words:
        if parse_tagged_word(token)[1] not in ['MAG', 'VA', 'VV']:
            continue
        try:
            cur_sims = cur_model.wv.most_similar(token, topn = 5000)
            for w in cur_sims:
                if parse_tagged_word(w[0])[1] == parse_tagged_word(token)[1] and w[1] >= min_cos_sim:
                    cur_negv_words.append(w)
                    if w[0] not in final_word_dict_keyset:
                        final_word_dict.update({w[0]: 0})
                    final_word_dict_keyset.add(w[0]) # final result
                    if final_word_dict[w[0]] < w[1]:
                        final_word_dict.update({w[0]: w[1]})
        except:
            continue

    wordset = set()
    for w in cur_negv_words:
        wordset.add(w[0])
    
    worddict = dict()
    for w in wordset:
        worddict[w] = []
    
    for w in cur_negv_words:
        worddict[w[0]].append(w[1])
    
    for key in worddict.keys():
        worddict[key] = max(worddict[key])
    
    print(list(worddict.items())[:10])
    print(len(list(worddict.items())))

# final_word_dict
print(list(final_word_dict.items())[:10])
print(len(list(final_word_dict.items())))

model_보건복지


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('걸어가/VV', 0.5836566686630249), ('아퍼/VA', 0.6126708984375), ('피/VV', 0.5067740082740784), ('춥/VA', 0.6441265940666199), ('높/VA', 0.5160777568817139), ('내뿜/VV', 0.5101895332336426), ('피해/VV', 0.5523408651351929), ('키오/VV', 0.6815026998519897), ('쏟/VV', 0.6168280839920044), ('하/VV', 0.5244807004928589)]
21
model_저출산/고령화대책
[('히/VV', 0.8718763589859009), ('각각/MAG', 0.5001537799835205), ('들어가/VV', 0.835469126701355), ('망하/VV', 0.9471550583839417), ('죽/VV', 0.925046980381012), ('모으/VV', 0.8744676113128662), ('그만큼/MAG', 0.9059579968452454), ('벌어지/VV', 0.9076480269432068), ('엄청/MAG', 0.8580238819122314), ('놓/VV', 0.9583693146705627)]
492
model_행정
[('많/VA', 0.5062159299850464), ('이/VV', 0.5017816424369812), ('안/MAG', 0.5239405632019043), ('진하/VA', 0.5939677953720093), ('사/VV', 0.5135031938552856), ('말리/VV', 0.5472069978713989), ('하/VV', 0.5507678985595703), ('있/VA', 0.5647357702255249), ('제발/MAG', 0.5796011686325073), ('살/VV', 0.5135031938552856)]
11
model_미래
[('이기/VV', 0.5344389081001282), ('

In [67]:
##### Create final result
import pandas
from pprint import pprint
final_res = list(final_word_dict.items())
final_res = [x for x in sorted(final_res, key=lambda x: x[1], reverse=True)]

pprint(final_res)

df = pandas.DataFrame(data = final_res)

df.to_csv("/content/gdrive/My Drive/senti/result/final_result.csv", mode = 'w', encoding = 'utf8', header = False, index = False)

[('스럽/VA', 0.9959872961044312),
 ('부리/VV', 0.9953685402870178),
 ('어찌하/VV', 0.9950308203697205),
 ('많이/MAG', 0.9947805404663086),
 ('제발/MAG', 0.9947375059127808),
 ('어떻/VV', 0.9946948289871216),
 ('먹/VV', 0.9943684935569763),
 ('추/VV', 0.9942302703857422),
 ('마시/VV', 0.9941911101341248),
 ('농사지/VV', 0.9939891695976257),
 ('이러/VV', 0.9936162829399109),
 ('물/VV', 0.9935330748558044),
 ('쉽/VA', 0.9934700727462769),
 ('그렇/VA', 0.9933821558952332),
 ('안타깝/VA', 0.9931901693344116),
 ('무섭/VA', 0.9931901097297668),
 ('밝히/VV', 0.9931792616844177),
 ('벌어지/VV', 0.9930709600448608),
 ('오늘/MAG', 0.9930700063705444),
 ('끝나/VV', 0.9926010370254517),
 ('아프/VA', 0.9925857782363892),
 ('싫/VA', 0.9925857186317444),
 ('생기/VV', 0.9925439357757568),
 ('떨어지/VV', 0.9925283193588257),
 ('미치/VV', 0.9925283193588257),
 ('아/VV', 0.9925094842910767),
 ('지내/VV', 0.9924914836883545),
 ('낳/VV', 0.9923053979873657),
 ('갑자기/MAG', 0.9922052621841431),
 ('기다리/VV', 0.992203950881958),
 ('자꾸/MAG', 0.9919577240943909),
 ('부